[Homework description](https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/cohorts/2024/04-deployment/homework.md) for Week 4 "Deployment"

In [ ]:
# %pip install pipenv
# !pipenv --python 3.11
# !pipenv install scikit-learn==1.5.0
# !pipenv install pandas
# !pipenv install pyarrow

In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.10.13


In [7]:
import os
import pickle
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [3]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

## Q1. Notebook
### What's the standard deviation of the predicted duration for this dataset?

In [6]:
y_pred.std()

6.247488852238703

## Q2. Preparing the output
### What's the size of the output file?

In [15]:
year = 2023
month = 2
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result = pd.DataFrame({
    'ride_id': df['ride_id'],  
    'prediction': y_pred
})

output_dir = 'output/yellow'
os.makedirs(output_dir, exist_ok=True)
output_file = f'{output_dir}/predictions-{year:04d}-{month:02d}.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [22]:
!du -h {output_file}

 65M	output/yellow/predictions-2023-02.parquet


In [24]:
file_size_mb = (os.path.getsize(output_file)) / 1024 / 1024
print('File size MB: ', round(file_size_mb, 2))

File size MB:  65.46


In [32]:
!ls -lh output/yellow/predictions-2023-02.parquet

-rw-r--r--  1 bilozorov  staff    65M Jun 12 18:02 output/yellow/predictions-2023-02.parquet


In [36]:
os.path.getsize(output_file) / (1024 * 1024)

65.46188354492188

## Q3. Creating the scoring script
### Which command you need to execute for that?

In [37]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 2069 bytes to starter.py


## Q4. Virtual environment
### What's the first hash for the Scikit-Learn dependency?

In [38]:
import json

# Path to the Pipfile.lock
pipfile_lock_path = 'Pipfile.lock'

# Read the Pipfile.lock
with open(pipfile_lock_path, 'r') as file:
    pipfile_lock_data = json.load(file)

# Find the Scikit-Learn dependency
scikit_learn_dependency = pipfile_lock_data['default'].get('scikit-learn', None)

# Check if Scikit-Learn is present
if scikit_learn_dependency:
    # Get the first hash
    first_hash = scikit_learn_dependency['hashes'][0]
    print(f"First hash for Scikit-Learn: {first_hash}")
else:
    print("Scikit-Learn dependency not found in Pipfile.lock")

First hash for Scikit-Learn: sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c


## Q5. Parametrize the script
### What's the mean predicted duration?

In [42]:
!pipenv run python starter.py yellow 2023 4

Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
The mean predicted duration is 14.292282936862449.


## Q6. Docker container
### What's the mean predicted duration for May 2023?



In [ ]:
!docker build -t homework4 .

In [45]:
!docker run --entrypoint python homework4 starter.py yellow 2023 5


The mean predicted duration is 0.19174419265916945 for 2023-05.


In [48]:
!docker run --rm homework4 yellow 2023 5

The mean predicted duration is 0.19174419265916945 for 2023-05.
